In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import PIL


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib



In [ ]:
path_string = "data/"
image_path = pathlib.Path(path_string)
image_count = len(list(image_path.glob('*/*.JPG')))
print(image_count)
data_dir = image_path


In [ ]:
batch_size = 32
img_height = 100
img_width = 100

In [ ]:
class_1_image = list(data_dir.glob('0/*'))
PIL.Image.open(str(class_1_image[0]))

In [ ]:
class_2_image = list(data_dir.glob('1/*'))
PIL.Image.open(str(class_2_image[0]))


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training", 
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)
print(num_classes)


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(img_height, img_width,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
    layers.experimental.preprocessing.RandomContrast(0.1),
  ]
)


In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")


In [ ]:
num_classes = len(class_names)

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
model.summary()


In [ ]:
epochs = 20

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
import cv2

cv2.namedWindow("TEST",0)

i = 0
for filename in os.listdir("demo"):
    img = cv2.imread(os.path.join("demo/",filename))
    #img = resize(img, (width, height))
    if img is None:
        print(os.path.join("demo/",filename))
        break
    else:
        
        image = img
        np_image_data = np.asarray(image)
        image = np.expand_dims(image, axis=0)
        predictions = model.predict(image)
        score = tf.nn.softmax(predictions)
        print("Auf dem Bild ist zu {:.2f}% eine {} zu sehen.".format(100 * np.max(score), class_names[np.argmax(score)]))    

        cv2.imshow("TEST", img)
        if cv2.waitKey(0) & 0xFF == ord('q'):    
            break


cv2.destroyAllWindows()

In [ ]:
import cv2

cv2.namedWindow("cam",0)

cap = cv2.VideoCapture(0)
  
while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = cap.read()
  
    # resize to 100x100
    cropped_full = frame[40:440, 120:620]
    down_points = (100, 100)
    cropped = cv2.resize(cropped_full, down_points,interpolation= cv2.INTER_LINEAR)

    # Display the resulting frame
    image = np.expand_dims(cropped, axis=0)
    predictions = model.predict(image)
    score = tf.nn.softmax(predictions)
    #print("Auf dem Bild ist zu {:.2f}% eine {} zu sehen.".format(100 * np.max(score), class_names[np.argmax(score)]))    

    cv2.putText(cropped_full, str("Auf dem Bild ist zu {:.2f}% eine {} zu sehen.".format(100 * np.max(score), class_names[np.argmax(score)])),
               (20, 30),
                1, 1, (0,0,0), 1)
    cv2.imshow('cam', cropped_full)

    # the 'q' button is set as the
    # quitting button you may use any
    # desired button of your choice
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
  


  
# After the loop release the cap object
cap.release()
cv2.destroyAllWindows()